In [11]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict

# Markov Chain with Extended State Space

In [12]:
def detect_sales_robust(df: pd.DataFrame, price_col: str = 'Price', 
                       method: str = 'rolling_quantile', **kwargs) -> pd.Series:

    if method == 'simple_mode':
        regular_price = df[price_col].mode().iloc[0]
        sale_threshold = regular_price * kwargs.get('threshold', 0.8)
        return np.where(df[price_col] < sale_threshold, 'S', 'N')
    
    elif method == 'rolling_quantile':
        window = kwargs.get('window', 60)
        quantile = kwargs.get('quantile', 0.8)
        threshold = kwargs.get('threshold', 0.8)
        
        normal_price = df[price_col].rolling(window=window, min_periods=10).quantile(quantile)
        normal_price = normal_price.fillna(method='bfill')
        
        sale_threshold = normal_price * threshold
        return np.where(df[price_col] < sale_threshold, 'S', 'N')


def identify_episodes(states: np.ndarray) -> List[Dict]:
    episodes = []
    
    if len(states) == 0:
        return episodes
    
    current_state = states[0]
    start_idx = 0
    
    for i in range(1, len(states)):
        if states[i] != current_state:
            episodes.append({
                'state': current_state,
                'start_idx': start_idx,
                'end_idx': i - 1,
                'duration': i - start_idx
            })
            
            current_state = states[i]
            start_idx = i
    
    episodes.append({
        'state': current_state,
        'start_idx': start_idx,
        'end_idx': len(states) - 1,
        'duration': len(states) - start_idx
    })
    
    return episodes


def calculate_state_boundaries(episodes: List[Dict], method: str = 'median_fraction') -> Dict:
    sale_durations = [ep['duration'] for ep in episodes if ep['state'] == 'S']
    no_sale_durations = [ep['duration'] for ep in episodes if ep['state'] == 'N']
    
    if method == 'median_fraction':
        if sale_durations:
            sale_median = np.median(sale_durations)
            sale_early_max = max(1, round(0.33 * sale_median))
            sale_mid_max = max(sale_early_max + 1, round(0.66 * sale_median))
            
            print(f"Sale median: {sale_median} days")
            print(f"Sale boundaries: 33% = {0.33 * sale_median:.1f} → {sale_early_max}, 66% = {0.66 * sale_median:.1f} → {sale_mid_max}")
        else:
            sale_early_max = 3
            sale_mid_max = 7
        
        if no_sale_durations:
            no_sale_median = np.median(no_sale_durations)
            no_sale_recent_max = max(1, round(0.33 * no_sale_median))
            no_sale_medium_max = max(no_sale_recent_max + 1, round(0.66 * no_sale_median))
            
            print(f"No-sale median: {no_sale_median} days")
            print(f"No-sale boundaries: 33% = {0.33 * no_sale_median:.1f} → {no_sale_recent_max}, 66% = {0.66 * no_sale_median:.1f} → {no_sale_medium_max}")
        else:
            no_sale_recent_max = 10
            no_sale_medium_max = 25
        
        boundaries = {
            'sale_early_max': sale_early_max,
            'sale_mid_max': sale_mid_max,
            'no_sale_recent_max': no_sale_recent_max,
            'no_sale_medium_max': no_sale_medium_max
        }
        
    elif method == 'manual':
        # Manual boundaries (easily adjustable)
        boundaries = {
            'sale_early_max': 3,
            'sale_mid_max': 7,
            'no_sale_recent_max': 10,
            'no_sale_medium_max': 25
        }
    
    return boundaries


def assign_extended_states(df: pd.DataFrame, episodes: List[Dict], 
                          boundaries: Dict) -> pd.Series:
    extended_states = np.full(len(df), '', dtype=object)
    
    for episode in episodes:
        start_idx = episode['start_idx']
        end_idx = episode['end_idx']
        state_type = episode['state']
        
        for day_idx in range(start_idx, end_idx + 1):
            days_into_episode = day_idx - start_idx + 1
            
            if state_type == 'S':
                if days_into_episode <= boundaries['sale_early_max']:
                    extended_states[day_idx] = 'S_early'
                elif days_into_episode <= boundaries['sale_mid_max']:
                    extended_states[day_idx] = 'S_mid'
                else:
                    extended_states[day_idx] = 'S_late'
            
            else:
                if days_into_episode <= boundaries['no_sale_recent_max']:
                    extended_states[day_idx] = 'N_recent'
                elif days_into_episode <= boundaries['no_sale_medium_max']:
                    extended_states[day_idx] = 'N_medium'
                else:
                    extended_states[day_idx] = 'N_old'
    
    return pd.Series(extended_states, index=df.index)


def process_product_data(filename: str, sale_detection_method: str = 'rolling_quantile',
                        boundary_method: str = 'percentile') -> Tuple[pd.DataFrame, Dict]:
    df = pd.read_csv(filename)
    df['Date'] = pd.to_datetime(df['Date'])
    
    df['simple_state'] = detect_sales_robust(df, method=sale_detection_method)
    
    episodes = identify_episodes(df['simple_state'].values)
    
    sale_episodes = [ep for ep in episodes if ep['state'] == 'S']
    no_sale_episodes = [ep for ep in episodes if ep['state'] == 'N']
    
    print(f"Found {len(sale_episodes)} sale episodes")
    print(f"Found {len(no_sale_episodes)} no-sale episodes")
    
    if sale_episodes:
        sale_durations = [ep['duration'] for ep in sale_episodes]
        print(f"Sale duration: min={min(sale_durations)}, median={np.median(sale_durations):.1f}, max={max(sale_durations)}")
    
    if no_sale_episodes:
        no_sale_durations = [ep['duration'] for ep in no_sale_episodes]
        print(f"No-sale duration: min={min(no_sale_durations)}, median={np.median(no_sale_durations):.1f}, max={max(no_sale_durations)}")
    
    boundaries = calculate_state_boundaries(episodes, method=boundary_method)
    print(f"State boundaries: {boundaries}")
    
    df['extended_state'] = assign_extended_states(df, episodes, boundaries)
    
    analysis_results = {
        'episodes': episodes,
        'boundaries': boundaries,
        'sale_episodes': len(sale_episodes),
        'no_sale_episodes': len(no_sale_episodes),
        'sale_durations': [ep['duration'] for ep in sale_episodes],
        'no_sale_durations': [ep['duration'] for ep in no_sale_episodes]
    }
    
    return df, analysis_results


filename = 'datasets/price_data_pesto.csv'

df_processed, results = process_product_data(
    filename, 
    sale_detection_method='rolling_quantile',
    boundary_method='median_fraction'
)

print("\nFirst 20 rows with extended states:")
print(df_processed[['Date', 'Price', 'simple_state', 'extended_state']].head(20))

print("\nExtended state distribution:")
print(df_processed['extended_state'].value_counts())

print("\nFirst 5 episodes:")
for i, episode in enumerate(results['episodes'][:5]):
    print(f"Episode {i+1}: {episode}")

Found 21 sale episodes
Found 22 no-sale episodes
Sale duration: min=3, median=7.0, max=8
No-sale duration: min=1, median=28.0, max=70
Sale median: 7.0 days
Sale boundaries: 33% = 2.3 → 2, 66% = 4.6 → 5
No-sale median: 28.0 days
No-sale boundaries: 33% = 9.2 → 9, 66% = 18.5 → 18
State boundaries: {'sale_early_max': 2, 'sale_mid_max': 5, 'no_sale_recent_max': 9, 'no_sale_medium_max': 18}

First 20 rows with extended states:
         Date  Price simple_state extended_state
0  2023-05-21   3.59            N       N_recent
1  2023-05-22   3.59            N       N_recent
2  2023-05-23   3.59            N       N_recent
3  2023-05-24   3.59            N       N_recent
4  2023-05-25   3.59            N       N_recent
5  2023-05-26   3.59            N       N_recent
6  2023-05-27   3.59            N       N_recent
7  2023-05-28   3.59            N       N_recent
8  2023-05-29   3.59            N       N_recent
9  2023-05-30   3.59            N       N_medium
10 2023-05-31   2.19            S  

C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')


In [13]:
from collections import Counter
import numpy as np
import pandas as pd

extended_states = df_processed['extended_state'].values
extended_transitions = [(extended_states[i], extended_states[i+1]) for i in range(len(extended_states)-1)]

all_states = ['S_early', 'S_mid', 'S_late', 'N_recent', 'N_medium', 'N_old']
state_to_idx = {state: i for i, state in enumerate(all_states)}

transition_counts = Counter(extended_transitions)

n_states = len(all_states)
transition_matrix = np.zeros((n_states, n_states))

for (from_state, to_state), count in transition_counts.items():
    from_idx = state_to_idx[from_state]
    to_idx = state_to_idx[to_state]
    transition_matrix[from_idx][to_idx] = count

for i in range(n_states):
    row_sum = transition_matrix[i].sum()
    if row_sum > 0:
        transition_matrix[i] = transition_matrix[i] / row_sum

transition_df = pd.DataFrame(
    transition_matrix,
    index=[f'From {state}' for state in all_states],
    columns=[f'To {state}' for state in all_states]
)

print(f'Extended Transition Matrix for product {filename}')
print(transition_df.round(3))

row_sums = transition_matrix.sum(axis=1)
print(f'\nRow sums (should be 1.0): {row_sums.round(3)}')

print(f'\nTransition counts:')
for (from_state, to_state), count in sorted(transition_counts.items()):
    print(f'{from_state} -> {to_state}: {count}')

def predict_sale_probability(transition_matrix, initial_state, days_ahead=10):
    if initial_state not in state_to_idx:
        raise ValueError(f"Invalid initial state: {initial_state}. Must be one of {all_states}")
    
    state_vector = np.zeros(len(all_states))
    initial_idx = state_to_idx[initial_state]
    state_vector[initial_idx] = 1.0
    
    forecast = []
    
    for day in range(1, days_ahead + 1):
        state_vector = state_vector @ transition_matrix
        
        sale_probability = state_vector[0] + state_vector[1] + state_vector[2]
        forecast.append(sale_probability)
    
    return forecast

print(f'\n=== MANUAL STATE PREDICTIONS ===')

initial_state = 'S_early'
days_ahead = 10
forecast = predict_sale_probability(transition_matrix, initial_state, days_ahead)

for day, day_forecast in enumerate(forecast, start=1):
    print(f"Day {day}: {day_forecast:.2%} chance of sale")

print(f'State distribution:')
print(df_processed['extended_state'].value_counts())

Extended Transition Matrix for product datasets/price_data_pesto.csv
               To S_early  To S_mid  To S_late  To N_recent  To N_medium  \
From S_early        0.500      0.50      0.000        0.000        0.000   
From S_mid          0.000      0.65      0.317        0.033        0.000   
From S_late         0.000      0.00      0.500        0.500        0.000   
From N_recent       0.011      0.00      0.000        0.886        0.103   
From N_medium       0.026      0.00      0.000        0.000        0.874   
From N_old          0.059      0.00      0.000        0.000        0.000   

               To N_old  
From S_early      0.000  
From S_mid        0.000  
From S_late       0.000  
From N_recent     0.000  
From N_medium     0.099  
From N_old        0.941  

Row sums (should be 1.0): [1. 1. 1. 1. 1. 1.]

Transition counts:
N_medium -> N_medium: 132
N_medium -> N_old: 15
N_medium -> S_early: 4
N_old -> N_old: 238
N_old -> S_early: 15
N_recent -> N_medium: 19
N_recent -> 

In [14]:
def multi_product_prediction(file_paths, initial_states, days_ahead=10):
    if len(file_paths) != len(initial_states):
        raise ValueError("Number of file paths must match number of initial states")
    
    product_results = {}
    all_forecasts = []
    
    for i, (file_path, initial_state) in enumerate(zip(file_paths, initial_states)):
        print(f"Processing Product {i+1}: {file_path}")
        print(f"Initial state: {initial_state}")
        print("-" * 50)
        
        try:
            df_processed, results = process_product_data(
                file_path, 
                sale_detection_method='rolling_quantile',
                boundary_method='median_fraction'
            )
            
            extended_states = df_processed['extended_state'].values
            extended_transitions = [(extended_states[j], extended_states[j+1]) 
                                  for j in range(len(extended_states)-1)]
            
            transition_counts = Counter(extended_transitions)
            
            transition_matrix = np.zeros((6, 6))
            
            for (from_state, to_state), count in transition_counts.items():
                from_idx = state_to_idx[from_state]
                to_idx = state_to_idx[to_state]
                transition_matrix[from_idx][to_idx] = count
            
            for row in range(6):
                row_sum = transition_matrix[row].sum()
                if row_sum > 0:
                    transition_matrix[row] = transition_matrix[row] / row_sum
            
            forecast = predict_sale_probability(transition_matrix, initial_state, days_ahead)
            
            product_results[f'Product_{i+1}'] = {
                'file_path': file_path,
                'initial_state': initial_state,
                'forecast': forecast,
                'transition_matrix': transition_matrix,
                'df_processed': df_processed,
                'results': results
            }
            
            all_forecasts.append(forecast)
            
            print(f"Forecast for Product {i+1}:")
            for day, prob in enumerate(forecast, start=1):
                print(f"  Day {day}: {prob:.2%}")
            print()
            
        except Exception as e:
            print(f"Error processing Product {i+1}: {e}")
            print()
            continue
    
    if all_forecasts:
        avg_forecast = []
        for day in range(days_ahead):
            day_probs = [forecast[day] for forecast in all_forecasts]
            avg_prob = sum(day_probs) / len(day_probs)
            avg_forecast.append(avg_prob)
        
        print("=== AVERAGE PROBABILITIES ===")
        print(f"Average probabilities for next {days_ahead} days:")
        
        for day, prob in enumerate(avg_forecast, start=1):
            print(f"  Day {day}: {prob:.2%}")
        
        return {
            'product_results': product_results,
            'avg_forecast': avg_forecast,
            'num_products': len(all_forecasts)
        }
    
    else:
        print("No products were successfully processed!")
        return None


def display_detailed_comparison(results):
    if not results:
        print("No results to display")
        return
    
    print("\n=== DETAILED PRODUCT COMPARISON ===")
    
    days_to_show = 10
    product_names = list(results['product_results'].keys())
    
    print(f"{'Day':<5}", end="")
    for product in product_names:
        print(f"{product:<12}", end="")
    print(f"{'Average':<10}")
    
    print("-" * (5 + 12 * len(product_names) + 10))
    
    for day in range(days_to_show):
        print(f"{day+1:<5}", end="")
        
        for product in product_names:
            forecast = results['product_results'][product]['forecast']
            prob = forecast[day] if day < len(forecast) else 0
            print(f"{prob:.1%}{'':>7}", end="")
        
        avg_prob = results['avg_forecast'][day] if day < len(results['avg_forecast']) else 0
        print(f"{avg_prob:.1%}")
    
    print(f"\nInitial states:")
    for product in product_names:
        initial_state = results['product_results'][product]['initial_state']
        print(f"  {product}: {initial_state}")

file_paths = [
    'datasets/price_data_pesto.csv',
    'datasets/price_data_chicken.csv',
    'datasets/price_data_coffee.csv',
    'datasets/price_data_butter.csv',
    'datasets/price_data_noodles.csv'
]

initial_states = [
    'S_early',
    'S_early',
    'S_early',
    'S_early',
    'S_early'
]

results = multi_product_prediction(file_paths, initial_states, days_ahead=10)

if results:
    display_detailed_comparison(results)

Processing Product 1: datasets/price_data_pesto.csv
Initial state: S_early
--------------------------------------------------
Found 21 sale episodes
Found 22 no-sale episodes
Sale duration: min=3, median=7.0, max=8
No-sale duration: min=1, median=28.0, max=70
Sale median: 7.0 days
Sale boundaries: 33% = 2.3 → 2, 66% = 4.6 → 5
No-sale median: 28.0 days
No-sale boundaries: 33% = 9.2 → 9, 66% = 18.5 → 18
State boundaries: {'sale_early_max': 2, 'sale_mid_max': 5, 'no_sale_recent_max': 9, 'no_sale_medium_max': 18}
Forecast for Product 1:
  Day 1: 100.00%
  Day 2: 98.33%
  Day 3: 88.52%
  Day 4: 73.92%
  Day 5: 58.51%
  Day 6: 44.74%
  Day 7: 33.64%
  Day 8: 25.35%
  Day 9: 19.52%
  Day 10: 15.68%

Processing Product 2: datasets/price_data_chicken.csv
Initial state: S_early
--------------------------------------------------
Found 23 sale episodes
Found 24 no-sale episodes
Sale duration: min=5, median=7.0, max=21
No-sale duration: min=7, median=14.0, max=70
Sale median: 7.0 days
Sale boundari

C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfil

In [15]:
def evaluate_practical_sale_prediction(file_paths, threshold=0.6):
    print("=== PRACTICAL SALE DURATION PREDICTION ===")
    print(f"Threshold: {threshold:.1%}")
    
    results = {}
    
    for i, file_path in enumerate(file_paths):
        product_name = file_path.split('/')[-1].replace('.txt', '').replace('price_data_', '').title()
        print(f"Evaluating {product_name}:")
        
        df_processed, analysis_results = process_product_data(
            file_path, 
            sale_detection_method='rolling_quantile',
            boundary_method='median_fraction'
        )
        
        actual_median_duration = int(np.median(analysis_results['sale_durations'])) if analysis_results['sale_durations'] else 7
        expected_remaining_days = actual_median_duration - 1 
        
        print(f"  Actual median total duration: {actual_median_duration} days")
        print(f"  Expected remaining days after today: {expected_remaining_days} days")
        
        extended_states = df_processed['extended_state'].values
        extended_transitions = [(extended_states[j], extended_states[j+1]) 
                              for j in range(len(extended_states)-1)]
        
        transition_counts = Counter(extended_transitions)
        transition_matrix = np.zeros((6, 6))
        
        for (from_state, to_state), count in transition_counts.items():
            from_idx = state_to_idx[from_state]
            to_idx = state_to_idx[to_state]
            transition_matrix[from_idx][to_idx] = count
        
        for row in range(6):
            row_sum = transition_matrix[row].sum()
            if row_sum > 0:
                transition_matrix[row] = transition_matrix[row] / row_sum
        
        if expected_remaining_days > 0:
            forecast = predict_sale_probability(transition_matrix, 'S_early', expected_remaining_days)
            
            predicted_pattern = [1 if prob >= threshold else 0 for prob in forecast]
            
            predicted_remaining_days = 0
            for day_prediction in predicted_pattern:
                if day_prediction == 1:
                    predicted_remaining_days += 1
                else:
                    break
            
        else:
            forecast = []
            predicted_pattern = []
            predicted_remaining_days = 0
        
        remaining_days_error = abs(predicted_remaining_days - expected_remaining_days)
        remaining_days_accuracy = 1 - (remaining_days_error / expected_remaining_days) if expected_remaining_days > 0 else 1.0
        
        if expected_remaining_days > 0:
            expected_pattern = [1] * expected_remaining_days
            day_accuracy = sum(1 for i, pred in enumerate(predicted_pattern) if i < len(expected_pattern) and pred == expected_pattern[i]) / expected_remaining_days
        else:
            day_accuracy = 1.0
        
        total_predicted_duration = 1 + predicted_remaining_days
        
        results[product_name] = {
            'actual_total_duration': actual_median_duration,
            'expected_remaining_days': expected_remaining_days,
            'predicted_remaining_days': predicted_remaining_days,
            'total_predicted_duration': total_predicted_duration,
            'remaining_days_error': remaining_days_error,
            'remaining_days_accuracy': remaining_days_accuracy,
            'day_accuracy': day_accuracy,
            'probabilities': forecast,
            'prediction_pattern': predicted_pattern,
            'threshold': threshold
        }
        
        print(f"  Predicted remaining days: {predicted_remaining_days} days")
        print(f"  Remaining days error: {remaining_days_error} days")
        print(f"  Remaining days accuracy: {remaining_days_accuracy:.3f}")
        print(f"  Day-by-day accuracy: {day_accuracy:.3f}")
        if forecast:
            print(f"  Daily probabilities: {[f'{p:.2f}' for p in forecast]}")
        print(f"  Prediction pattern: {predicted_pattern}")
    
    all_remaining_errors = [r['remaining_days_error'] for r in results.values()]
    all_remaining_accuracies = [r['remaining_days_accuracy'] for r in results.values()]
    all_day_accuracies = [r['day_accuracy'] for r in results.values()]
    
    print("=== OVERALL RESULTS ===")
    print(f"Average remaining days error: {np.mean(all_remaining_errors):.2f} days")
    print(f"Average remaining days accuracy: {np.mean(all_remaining_accuracies):.3f}")
    print(f"Average day-by-day accuracy: {np.mean(all_day_accuracies):.3f}")
    
    return results

practical_results = evaluate_practical_sale_prediction(file_paths, threshold=0.7)

=== PRACTICAL SALE DURATION PREDICTION ===
Threshold: 70.0%
Evaluating Pesto.Csv:
Found 21 sale episodes
Found 22 no-sale episodes
Sale duration: min=3, median=7.0, max=8
No-sale duration: min=1, median=28.0, max=70
Sale median: 7.0 days
Sale boundaries: 33% = 2.3 → 2, 66% = 4.6 → 5
No-sale median: 28.0 days
No-sale boundaries: 33% = 9.2 → 9, 66% = 18.5 → 18
State boundaries: {'sale_early_max': 2, 'sale_mid_max': 5, 'no_sale_recent_max': 9, 'no_sale_medium_max': 18}
  Actual median total duration: 7 days
  Expected remaining days after today: 6 days
  Predicted remaining days: 4 days
  Remaining days error: 2 days
  Remaining days accuracy: 0.667
  Day-by-day accuracy: 0.667
  Daily probabilities: ['1.00', '0.98', '0.89', '0.74', '0.59', '0.45']
  Prediction pattern: [1, 1, 1, 1, 0, 0]
Evaluating Chicken.Csv:
Found 23 sale episodes
Found 24 no-sale episodes
Sale duration: min=5, median=7.0, max=21
No-sale duration: min=7, median=14.0, max=70
Sale median: 7.0 days
Sale boundaries: 33% =

C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfill')
C:\Users\pgaug\AppData\Local\Temp\ipykernel_7356\93707491.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normal_price = normal_price.fillna(method='bfil